# CNN

## MNIST 숫자 분류를 위한 CNN 분류기 구현 

In [1]:
import tensorflow as tf

In [2]:
# MNIST 데이터 다운로드
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/",one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# CNN 모델 정의 
def build_CNN_classifier(x): 
    # MNIST 데이터를 3차원 형태로 reshape MNIST 데이터는 grayscale 이미지이기 때문에 3번째 차원(컬러채널)의 값은 1이다.
    x_image = tf.reshape(x,[-1,28,28,1])
    
    # 1번째 컨볼루션 층
    # 5 x 5 Kernel size 를 가진 32개의 filter를 적용
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,1,32],stddev = 5e-2))
    b_conv1 = tf.Variable(tf.constant(0.1,shape=[32]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1],padding='SAME')+ b_conv1)
    
    # 1번째 풀링층
    # Max Pooling을 이용해서 이미지의 크기를 1/2로 downsample한다
    # 28 x 28 x 32 -> 14 x 14 x 32
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    # 2번째 컨볼루션층 
    # 5 x 5 kernel size를 가진 64개의 filter를 적용
    # 14 x 14 x 32 -> 14 x 14 x 64
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,32,64], stddev=5e-2))
    b_conv2 = tf.Variable(tf.constant(0.1,shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1,W_conv2,strides=[1,1,1,1],padding='SAME')+ b_conv2)
    
    # 2번째 풀링층 
    # Max pooling 을 이용해서 이미지의 크기를 1/2로 downsample
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    # 완전 연결층
    # 7 x 7 크기를 가진 64개의 activation map을 1024개의 특징들로 변환 
    # 7 x 7 x 64(3136)-> 1024
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[7*7*64,1024],stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1,shape=[1024]))
    h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
    
    # 출력층
    # 1024 개의 특징들(feature)을 10개의 클래스(One-hot Encoding으로 표현된 숫자 0~9)로 변환
    # 1024 -> 10 
    W_output = tf.Variable(tf.truncated_normal(shape=[1024,10],stddev=5e-2))
    b_output = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1, W_output)+ b_output
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

In [3]:
# 인풋, 아웃풋 데이터를 받기 위한 플레이스 홀더 정의 
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [8]:
# CNN 선언
y_pred, logits = build_CNN_classifier(x)

In [9]:
# Cross entropy 를 손실 함수(loss function)으로 정의하고 옵티마이저를 정의 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
# 정확도 계산
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
# 세션을 열어 실제 학습 진행
with tf.Session() as sess : 
    sess.run(tf.global_variables_initializer())
    
    # 10000 step 만큼 최적화 수행
    for i in range(10000): 
        # 50 개씩 MNIST 데이터 불러온다. 
        batch = mnist.train.next_batch(50)
        # 100 step 마다 training 데이터셋에 대한 정확도 출력
        if i % 100 == 0 : 
            train_accuracy = accuracy.eval(feed_dict={x:batch[0],y:batch[1]})
            print("반복(Epoch): %d, 트레이닝 데이터 정확도 : %f" % (i, train_accuracy))
        # 옵티마이저를 실행해 파라미터를 한 스텝 업데이트
        sess.run([train_step], feed_dict={x:batch[0],y:batch[1]})
        
    # 학습이 끝나면 테스트 데이터에 대한 정확도를 출력 
    print("테스트 데이터 정확도: %f" % accuracy.eval(feed_dict={x:mnist.test.images, y:mnist.test.labels}))

반복(Epoch): 0, 트레이닝 데이터 정확도 : 0.140000
반복(Epoch): 100, 트레이닝 데이터 정확도 : 0.780000
반복(Epoch): 200, 트레이닝 데이터 정확도 : 0.940000
반복(Epoch): 300, 트레이닝 데이터 정확도 : 0.940000
반복(Epoch): 400, 트레이닝 데이터 정확도 : 0.900000
반복(Epoch): 500, 트레이닝 데이터 정확도 : 0.960000
반복(Epoch): 600, 트레이닝 데이터 정확도 : 0.880000
반복(Epoch): 700, 트레이닝 데이터 정확도 : 1.000000
반복(Epoch): 800, 트레이닝 데이터 정확도 : 0.960000
반복(Epoch): 900, 트레이닝 데이터 정확도 : 0.920000
반복(Epoch): 1000, 트레이닝 데이터 정확도 : 0.940000
반복(Epoch): 1100, 트레이닝 데이터 정확도 : 0.980000
반복(Epoch): 1200, 트레이닝 데이터 정확도 : 0.980000
반복(Epoch): 1300, 트레이닝 데이터 정확도 : 0.960000
반복(Epoch): 1400, 트레이닝 데이터 정확도 : 1.000000
반복(Epoch): 1500, 트레이닝 데이터 정확도 : 0.980000
반복(Epoch): 1600, 트레이닝 데이터 정확도 : 0.940000
반복(Epoch): 1700, 트레이닝 데이터 정확도 : 0.980000
반복(Epoch): 1800, 트레이닝 데이터 정확도 : 0.920000
반복(Epoch): 1900, 트레이닝 데이터 정확도 : 1.000000
반복(Epoch): 2000, 트레이닝 데이터 정확도 : 0.980000
반복(Epoch): 2100, 트레이닝 데이터 정확도 : 1.000000
반복(Epoch): 2200, 트레이닝 데이터 정확도 : 0.980000
반복(Epoch): 2300, 트레이닝 데이터 정확도 : 0.960000
반복(Epoch): 2400, 트레이닝 데이터 정확

## CNN을 이용한 CIFAR-10 이미지 분류기 구현 

In [6]:
import numpy as np

In [7]:
# CIFAR-10 데이터를 다운로드 받기 위한 keras의 helper함수인 load_data함수를 임포트 
from tensorflow.keras.datasets.cifar10 import load_data


In [8]:
# 다음 배치를 읽어오기 위한 next_batch 유틸리티 함수를 정의 
def next_batch(num, data, labels): 
    '''
    'num' 개수 만큼의 랜덤한 샘플들과 레이블들을 리턴한다. 
    '''
    idx = np.arange(0,len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


In [10]:
# CNN 모델 정의 
def build_CNN_classifier(x): 
    # 입력 이미지
    x_image = x
    
    # 1번째 컨볼루션층 - 하나의 RCB 이미지를 64개의 특징들(feature)으로 맵핑(maping)한다.
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,3,64],stddev = 5e-2))
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides = [1,1,1,1],padding='SAME') + b_conv1)
    
    # 1번째 풀링층
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
    
    # 2번째 컨볼루션층 - 64개의 특징들을 64개의 특징들로 맵핑한다. 
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,64,64],stddev=5e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1],padding='SAME') + b_conv2)
    
    # 2번째 풀링층 
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
    
    # 3번째 컨볼루션층 
    W_conv3 = tf.Variable(tf.truncated_normal(shape=[3,3,64,128],stddev=5e-2))
    b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1,1,1,1],padding='SAME')+b_conv3)
    
    # 4번째 컨볼루션층 
    W_conv4 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128],stddev=5e-2))
    b_conv4 = tf.Variable(tf.constant(0.1,shape=[128]))
    h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1,1,1,1],padding='SAME')+b_conv4)
    
    # 5번째 컨볼루션층 
    W_conv5 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128],stddev=5e-2))
    b_conv5 = tf.Variable(tf.constant(0.1,shape=[128]))
    h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1,1,1,1],padding='SAME')+b_conv5)
    
    # 완전 연결층 1 - 2번의 downsampling 이후에, 우리의 32 x 32 이미지는 8x8x128 특징맵(feature map)이 된다. 
    # 이를 384개의 특징들로 맵핑한다. 
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*128, 384], stddev = 5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))
    
    h_conv5_flat = tf.reshape(h_conv5,[-1,8*8*128])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1)+b_fc1)
    
    # 드롭아웃 - 모델의 복잡도를 컨트롤. 특징들의 co-adapting을 방지 
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    # 완전 연결층 2 - 384개의 특징들을 10개의 클래스로 맵핑
    W_fc2 = tf.Variable(tf.truncated_normal(shape=[384,10],stddev=5e-2))
    b_fc2 = tf.Variable(tf.constant(0.1,shape=[10]))
    logits = tf.matmul(h_fc1_drop,W_fc2)+b_fc2
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

In [11]:
x = tf.placeholder(tf.float32, shape=[None, 32,32,3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

In [12]:
# CIFAR-10 데이터를 다운로드하고 데이터를 불러온다. 
(x_train, y_train), (x_test, y_test) = load_data()
# scalar 형태의 레이블(0~9)을 one-hot encoding 형태로 반환
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis = 1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test,10),axis = 1)

In [13]:
# CNN 그래프 생성
y_pred, logits = build_CNN_classifier(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
# Cross entropy를 비용 함수로 정의, RMSPropOptimizer 을 이용해서 비용 함수를 최소화 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
# 정확도 계산
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# 세션을 열어 실제 학습 진행 
with tf.Session() as sess : 
    sess.run(tf.global_variables_initializer())
    
    # 10000 step 만큼 최적화 수행
    for i in range(10000): 
        
        batch = next_batch(128, x_train, y_train_one_hot.eval())
        # 100 step 마다 training 데이터셋에 대한 정확도와 loss 출력
        if i % 100 == 0 : 
            train_accuracy = accuracy.eval(feed_dict={x:batch[0],y:batch[1], keep_prob:1.0})
            loss_print = loss.eval(feed_dict={x:batch[0],y:batch[1], keep_prob:1.0})
            
            print("반복(Epoch): %d, 트레이닝 데이터 정확도 : %f, 손실 함수(loss):%f" % (i, train_accuracy,loss_print))
            
        # 20% 확률의 드롭아웃을 이용해서 학습을 진행 
        sess.run([train_step], feed_dict={x:batch[0],y:batch[1], keep_prob : 0.8})
        
    # 학습이 끝나면 테스트 데이터에 대한 정확도를 출력 
    test_accuracy = 0.0
    for i in range(10): 
        test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
        test_accuracy = test_accuracy + accuracy.eval(feed_dict={x:test_batch[0],y:test_batch[1],keep_prob:1.0})
        test_accuracy=test_accuracy / 10 ;
    
        print("테스트 데이터 정확도: %f" % test_accuracy)

반복(Epoch): 0, 트레이닝 데이터 정확도 : 0.093750, 손실 함수(loss):137.367584
반복(Epoch): 100, 트레이닝 데이터 정확도 : 0.195312, 손실 함수(loss):2.230953
반복(Epoch): 200, 트레이닝 데이터 정확도 : 0.140625, 손실 함수(loss):2.415778
반복(Epoch): 300, 트레이닝 데이터 정확도 : 0.265625, 손실 함수(loss):1.915620
반복(Epoch): 400, 트레이닝 데이터 정확도 : 0.343750, 손실 함수(loss):1.837606
반복(Epoch): 500, 트레이닝 데이터 정확도 : 0.414062, 손실 함수(loss):1.570273
반복(Epoch): 600, 트레이닝 데이터 정확도 : 0.500000, 손실 함수(loss):1.399151


In [5]:
print(tf.__version__)

1.15.0
